In [2]:
import os
import numpy as np
import pandas as pd
import nibabel as nib

from nilearn import plotting

In [11]:
# Creating folder path for each folder in directory

folder_contents_path = ['data/MICCAI_BraTS_2019_Data_Training/HGG/' + str(name) for name in os.listdir('data/MICCAI_BraTS_2019_Data_Training/HGG')]

In [64]:
img_data = []

for folder_path in folder_contents_path:
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + str(file)
        
        if '.nii.gz' in file_path:
            img = nib.load(file_path)
            img_data.append(img.get_data())
        
img_data

ImageFileError: Empty file: 'data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_ANG_1/BraTS19_CBICA_ANG_1_t1ce.nii.gz'

In [52]:

nib.load('data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_ANG_1/BraTS19_CBICA_ANG_1_flair.nii.gz')



In [53]:

nib.load('data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_ANG_1/BraTS19_CBICA_ANG_1_t1ce.nii.gz')



ImageFileError: Empty file: 'data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_ANG_1/BraTS19_CBICA_ANG_1_t1ce.nii.gz'